# Normalization

# K-Means to classify users

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spotipy
import sys
import pprint
import warnings
warnings.filterwarnings("ignore")
import math
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id='653cfed9a26e46dca3e3e211ded03c0f',\
                                                      client_secret='a14b7886714c41c383458b26434d60cc')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
import spotipy
import json
import time
import copy
import scipy.sparse as ssp
from collections import Counter, defaultdict
import tqdm
from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix
from scipy.stats import pearsonr
from __future__ import print_function    
import time
from scipy import stats as sss
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import roc_auc_score as auc
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.neural_network import MLPRegressor
import pickle
from scipy import io
from sklearn.cluster import KMeans
from sklearn.neural_network import BernoulliRBM
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor

In [8]:
# #read
train = pd.read_csv('/Users/osx/Desktop/Warner Music Dissertation/Code/train_supervised.csv')
train = train.drop('Unnamed: 0',axis=1)

In [9]:
y_label = train['ratings']
x_input = train.drop('ratings',axis=1)
x_user = x_input.drop('customer_id',axis=1)

In [10]:
age = pd.DataFrame(2017-train['birth_year'].values)
train = pd.concat([train,age],axis=1)
train.rename(columns={0:'age'},inplace=True)
train.drop('birth_year',axis=1,inplace=True)

In [11]:
train.sort_values('customer_id',inplace=True)
train = train.reset_index(drop=True)

In [12]:
train.head(10)

,customer_id,artist_name,track_name,album_name,ratings,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,age
0,00000255e47132304859a31172d40a34,Ed Sheeran,Nina,x,1,0.175000,0.791,225533.0,0.669,0.000000,6.0,0.3300,-7.189,0.0,0.1460,92.013,4.0,0.8070,26
1,00000255e47132304859a31172d40a34,Ed Sheeran,The Man,x,1,0.034600,0.526,254880.0,0.772,0.000000,11.0,0.2140,-7.229,0.0,0.0479,85.876,4.0,0.5110,26
2,0000046be044aa6c953ff75558237c54,Charli XCX,Doing It (feat. Rita Ora),SUCKER,1,0.028200,0.611,228520.0,0.947,0.000000,4.0,0.3030,-1.538,0.0,0.0558,105.940,4.0,0.5710,19
3,0000046be044aa6c953ff75558237c54,David Guetta,What I did for Love (feat. Emeli Sandé),Listen,1,0.028200,0.611,228520.0,0.947,0.000000,4.0,0.3030,-1.538,0.0,0.0558,105.940,4.0,0.5710,19
4,0000046be044aa6c953ff75558237c54,Flo Rida,GDFR (feat. Sage The Gemini & Lookas),GDFR (feat. Sage The Gemini & Lookas),1,0.000704,0.657,190186.0,0.827,0.005340,5.0,0.0650,-4.036,1.0,0.0734,145.889,4.0,0.6980,19
5,0000046be044aa6c953ff75558237c54,Fetty Wap,Trap Queen,Fetty Wap,1,0.053300,0.791,221858.0,0.698,0.000000,4.0,0.3470,-7.326,0.0,0.0915,147.965,4.0,0.7680,19
6,00000a8eb0520ce9081c5a6767e9ebb9,Ray LaMontagne,Let It Be Me,Gossip in The Grain,1,0.581000,0.239,185907.0,0.282,0.000009,5.0,0.1180,-13.067,1.0,0.0313,94.518,4.0,0.0997,22
7,00000a8eb0520ce9081c5a6767e9ebb9,Ed Sheeran,Afire Love,x,1,0.954000,0.481,282957.0,0.325,0.890000,2.0,0.0913,-15.918,0.0,0.0388,179.969,4.0,0.3300,22
8,00000a8eb0520ce9081c5a6767e9ebb9,Gorillaz,Feel Good Inc,Demon Days,1,0.143000,0.737,224235.0,0.753,0.000064,6.0,0.0896,-6.562,1.0,0.1110,138.067,4.0,0.8380,22
9,00000a8eb0520ce9081c5a6767e9ebb9,Major Lazer,Lean On (feat. MØ & DJ Snake),Lean On (feat. MØ & DJ Snake),2,0.143000,0.737,224235.0,0.753,0.000064,6.0,0.0896,-6.562,1.0,0.1110,138.067,4.0,0.8380,22


In [13]:
user, index = np.unique(train['customer_id'].values,True)
customer = np.split(train['customer_id'].values,index[1:])

acout = np.split(train['acousticness'].values,index[1:])
rat = np.split(train['ratings'].values,index[1:])
danceability = np.split(train['danceability'].values,index[1:])
duration = np.split(train['duration_ms'].values,index[1:])
energy = np.split(train['energy'].values,index[1:])
instrumentalness = np.split(train['instrumentalness'].values,index[1:])
key = np.split(train['key'].values,index[1:])
liveness = np.split(train['liveness'].values,index[1:])
loudness = np.split(train['loudness'].values,index[1:])
mode = np.split(train['mode'].values,index[1:])
speechiness = np.split(train['speechiness'].values,index[1:])
tempo = np.split(train['tempo'].values,index[1:])
time_signature = np.split(train['time_signature'].values,index[1:])
valence = np.split(train['valence'].values,index[1:])
age = np.split(train['age'].values,index[1:])

In [14]:
cluster_X = []
for i in tqdm.tqdm(range(20000)):
    rat_ = rat[i]
    acout_ = np.sum((acout[i]*rat[i])/np.sum(rat[i]))
    danceability_ = np.sum((danceability[i]*rat[i])/np.sum(rat[i]))
    duration_ = np.sum((duration[i]*rat[i])/np.sum(rat[i]))
    energy_ = np.sum((energy[i]*rat[i])/np.sum(rat[i]))
    instrumentalness_ = np.sum((instrumentalness[i]*rat[i])/np.sum(rat[i]))
    key_ = np.sum((key[i]*rat[i])/np.sum(rat[i]))
    liveness_ = np.sum((liveness[i]*rat[i])/np.sum(rat[i]))
    loudness_ = np.sum((loudness[i]*rat[i])/np.sum(rat[i]))
    mode_ = np.sum((mode[i]*rat[i])/np.sum(rat[i]))
    speechiness_ = np.sum((speechiness[i]*rat[i])/np.sum(rat[i]))
    tempo_ = np.sum((tempo[i]*rat[i])/np.sum(rat[i]))
    time_signature_ = np.sum((time_signature[i]*rat[i])/np.sum(rat[i]))
    valence_ = np.sum((valence[i]*rat[i])/np.sum(rat[i]))
    age_ = np.sum((age[i]*rat[i])/np.sum(rat[i]))
    cluster_X.append(np.array([acout_,danceability_,duration_,energy_,instrumentalness_,key_,liveness_,loudness_,mode_,\
                               speechiness_,tempo_,time_signature_,valence_,age_]))

100%|██████████| 20000/20000 [00:03<00:00, 6653.89it/s]


In [15]:
kmeans = KMeans(n_clusters=6, random_state=0).fit(cluster_X)

In [16]:
group_1 = np.array(np.where(kmeans.labels_==0))[0]
group_2 = np.array(np.where(kmeans.labels_==1))[0]
group_3 = np.array(np.where(kmeans.labels_==2))[0]
group_4 = np.array(np.where(kmeans.labels_==3))[0]
group_5 = np.array(np.where(kmeans.labels_==4))[0]
group_6 = np.array(np.where(kmeans.labels_==5))[0]
# group_7 = np.array(np.where(kmeans.labels_==6))[0]
# group_8 = np.array(np.where(kmeans.labels_==7))[0]
# group_9 = np.array(np.where(kmeans.labels_==8))[0]
# group_10 = np.array(np.where(kmeans.labels_==9))[0]

In [17]:
k=0
for j in tqdm.tqdm([group_1,group_2,group_3,group_4,group_5,group_6]):
    k+=1
    group_list = []
    for i in j:
        group_list.append(customer[i][0])
    if k==1:
        group_df_1 = train[train.customer_id.isin(group_list)]
    if k==2:
        group_df_2 = train[train.customer_id.isin(group_list)]
    if k==3:
        group_df_3 = train[train.customer_id.isin(group_list)]
    if k==4:
        group_df_4 = train[train.customer_id.isin(group_list)]
    if k==5:
        group_df_5 = train[train.customer_id.isin(group_list)]
    if k==6:
        group_df_6 = train[train.customer_id.isin(group_list)]
#     if k==7:
#         group_df_7 = train[train.customer_id.isin(group_list)]
#     if k==8:
#         group_df_8 = train[train.customer_id.isin(group_list)]
#     if k==9:
#         group_df_9 = train[train.customer_id.isin(group_list)]
#     if k==10:
#         group_df_10 = train[train.customer_id.isin(group_list)]

100%|██████████| 6/6 [00:00<00:00, 77.55it/s]


In [18]:
y_1 = group_df_1['ratings']
customer_1=group_df_1['customer_id']
y_2 = group_df_2['ratings']
customer_2=group_df_2['customer_id']
y_3 = group_df_3['ratings']
customer_3=group_df_3['customer_id']
y_4 = group_df_4['ratings']
customer_4=group_df_4['customer_id']
y_5 = group_df_5['ratings']
customer_5=group_df_5['customer_id']
y_6 = group_df_6['ratings']
customer_6=group_df_6['customer_id']
y = train['ratings']
# y_7 = group_df_7['ratings']
# customer_7=group_df_7['customer_id']
# y_8 = group_df_8['ratings']
# customer_8=group_df_8['customer_id']
# y_9 = group_df_9['ratings']
# customer_9=group_df_9['customer_id']
# y_10 = group_df_10['ratings']
# customer_10=group_df_10['customer_id']

features_to_drop=['customer_id', 'artist_name', 'track_name', 'album_name','ratings']
group_df_1.drop(features_to_drop,axis=1,inplace=True)
group_df_2.drop(features_to_drop,axis=1,inplace=True)
group_df_3.drop(features_to_drop,axis=1,inplace=True)
group_df_4.drop(features_to_drop,axis=1,inplace=True)
group_df_5.drop(features_to_drop,axis=1,inplace=True)
group_df_6.drop(features_to_drop,axis=1,inplace=True)
train.drop(features_to_drop,axis=1,inplace=True)
# group_df_7.drop(features_to_drop,axis=1,inplace=True)
# group_df_8.drop(features_to_drop,axis=1,inplace=True)
# group_df_9.drop(features_to_drop,axis=1,inplace=True)
# group_df_10.drop(features_to_drop,axis=1,inplace=True)

In [13]:
# kmeans.labels_.tolist()
# kmeans.predict([[0, 0], [4, 4]])
# kmeans.cluster_centers_

# Supervised Learning

##### Random Forest Regressor

In [ ]:
param_to_tuned = {'max_depth':[3,6,8],
                  'n_estimators':[500,700,300],
                  'min_samples_split':[3,5,10],
                  'min_samples_leaf':[1,3],
                  'min_weight_fraction_leaf':[0],
                  'max_leaf_nodes':[None],
                  'oob_score':[False],
                  'bootstrap':[True],
                  'class_weight':['balanced']}

start = time.time()

rfr_model = RandomForestRegressor()    
rfr_Model = GridSearchCV(rfc_model, param_grid = param_to_tuned,cv=10,refit=True,return_train_score=True)
RFR = rfr_Model.fit(train, y)

RFR_1 = rfr_Model.fit(group_df_1,y_1)
RFR_2 = rfr_Model.fit(group_df_2,y_2)
RFR_3 = rfr_Model.fit(group_df_3,y_3)
RFR_4 = rfr_Model.fit(group_df_4,y_4)
RFR_5 = rfr_Model.fit(group_df_5,y_5)
RFR_6 = rfr_Model.fit(group_df_6,y_6)

delta = time.time() - start

print('model training time:',round((time.time()-t)/60,3),'minutes\n')
print('optimal parameter:')
print('max_depth:',rfc_Model.best_params_['max_depth'])
print('n_estimators:',rfc_Model.best_params_['n_estimators'])
print('min_samples_split:',rfc_Model.best_params_['min_samples_split'])
print('min_samples_leaf:',rfc_Model.best_params_['min_samples_leaf'])
print ("model trained finished in %.2f seconds" % (delta,))

In [13]:
start = time.time()

RFR = RandomForestRegressor(max_depth=5,n_estimators=300)
RFR_1 = RFR.fit(group_df_1,y_1)
RFR_2 = RFR.fit(group_df_2,y_2)
RFR_3 = RFR.fit(group_df_3,y_3)
RFR_4 = RFR.fit(group_df_4,y_4)
RFR_5 = RFR.fit(group_df_5,y_5)
RFR_6 = RFR.fit(group_df_6,y_6)
# RFR_7 = RFR.fit(group_df_7,y_7)
# RFR_8 = RFR.fit(group_df_8,y_8)
# RFR_9 = RFR.fit(group_df_9,y_9)
# RFR_10 = RFR.fit(group_df_10,y_10)
delta = time.time() - start
print ("model trained finished in %.2f seconds" % (delta,))

model trained finished in 60.48 seconds


##### linear SVM Regressor

In [ ]:
param_to_tuned = {'kernel':['rbf','linear','sigmoid'], 
                  'degree':[3], 'gamma':['auto'],
                  'max_iter':[-1,'solver']}

start = time.time()

SVR_model = svm.SVR()
svr_Model = GridSearchCV(SVR_model, param_grid = param_to_tuned,cv=10,refit=True,return_train_score=True)
SVR = svr_Model.fit(train, y)

SVR_1 = svr_Model.fit(group_df_1,y_1)
SVR_2 = svr_Model.fit(group_df_2,y_2)
SVR_3 = svr_Model.fit(group_df_3,y_3)
SVR_4 = svr_Model.fit(group_df_4,y_4)
SVR_5 = svr_Model.fit(group_df_5,y_5)
SVR_6 = svr_Model.fit(group_df_6,y_6)

delta = time.time() - start
print ("model trained finished in %.2f seconds" % (delta,))

##### XGBoost Regressor

In [32]:
param_to_tuned = {'max_depth':[3,6,8],
                  'seed':[1337],
                  'silent':[1],
                  'n_estimators':[500,700,300],
                  'learning_rate':np.linspace(0.01,0.6,6),
                  'subsample':[0.1,0.8],
                  'colsample_bytree':[0.7],
                  'gamma':[0,1,0.5],
                  'min_child_weight':[0,0.5,1.5]}

start = time.time()

xgb_model = xgb.XGBRegressor()    
XGB_Model = GridSearchCV(xgb_model, param_grid = param_to_tuned, scoring='neg_mean_absolute_error',\
                         cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=None),\
                          refit=True,return_train_score=True)
t = time.time()
XGB = XGB_Model.fit(train, y)

# SVR_1 = XGB_Model.fit(group_df_1,y_1)
# SVR_2 = XGB_Model.fit(group_df_2,y_2)
# SVR_3 = XGB_Model.fit(group_df_3,y_3)
# SVR_4 = XGB_Model.fit(group_df_4,y_4)
# SVR_5 = XGB_Model.fit(group_df_5,y_5)
# SVR_6 = XGB_Model.fit(group_df_6,y_6)

delta = time.time() - start
print ("model trained finished in %.2f seconds" % (delta,))

KeyboardInterrupt: 

In [39]:
xgb.XGBRegressor.score

<function sklearn.base.RegressorMixin.score>

##### Ridge Regressor

In [ ]:
param_to_tuned = {'alpha':[1.0,0.5,0.01],
                  'normalize':[False,True],
                  }

start = time.time()

ridge_model = Ridge(alpha=1.0, fit_intercept=True, normalize=False,\
                    copy_X=True, max_iter=None, tol=0.001, \
                    solver=’auto’, random_state=None)

   
ridge_Model = GridSearchCV(ridge_model, param_grid = param_to_tuned, scoring='rmse',\
                         cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=None),\
                          refit=True,return_train_score=True)
t = time.time()
Ridge = ridge_Model.fit(train, y)

Ridge_1 = ridge_Model.fit(group_df_1,y_1)
Ridge_2 = ridge_Model.fit(group_df_2,y_2)
Ridge_3 = ridge_Model.fit(group_df_3,y_3)
Ridge_4 = ridge_Model.fit(group_df_4,y_4)
Ridge_5 = ridge_Model.fit(group_df_5,y_5)
Ridge_6 = ridge_Model.fit(group_df_6,y_6)

delta = time.time() - start
print ("model trained finished in %.2f seconds" % (delta,))

##### Nerual Network

In [366]:
param_to_tuned = {'n_components':[256,150,300],
                  'learning_rate':[0.1,0.5,0.6],
                  'batch_size':[10,50,5],
                  'n_iter':[10,6,15],
                  }


start = time.time()

RBM_model = BernoulliRBM()
RBM_Model = GridSearchCV(RBM_model, param_grid = param_to_tuned, scoring='rmse',\
                         cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=None),\
                          refit=True,return_train_score=True)
rbm = RBM_Model.fit(train,y)

rbm_1 = RBM_Model.fit(group_df_1,y_1)
rbm_2 = RBM_Model.fit(group_df_2,y_2)
rbm_3 = RBM_Model.fit(group_df_3,y_3)
rbm_4 = RBM_Model.fit(group_df_4,y_4)
rbm_5 = RBM_Model.fit(group_df_5,y_5)
rbm_6 = RBM_Model.fit(group_df_6,y_6)

delta = time.time() - start
print ("model trained finished in %.2f seconds" % (delta,))

model trained finished in 22.92 seconds


In [ ]:
param_to_tuned = {'hidden_layer_sizes':[(100,200,300),(150,300,500)],
                  'activation':['relu','logistic'],
                  'learning_rate_init':[0.0001,0.0005,0.001],
                  'max_iter':[200,300,500],
                  }

start = time.time()

NN_model = MLPRegressor(solver=’adam’,\
                        alpha=0.0001, batch_size=’auto’,\
                        shuffle=True,random_state=None, tol=0.0001, verbose=False, warm_start=False)
NN_Model = GridSearchCV(NN_model, param_grid = param_to_tuned, scoring='rmse',\
                         cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=None),\
                          refit=True,return_train_score=True)
nn = NN_Model.fit(train,y)

nn_1 = NN_Model.fit(group_df_1,y_1)
nn_2 = NN_Model.fit(group_df_2,y_2)
nn_3 = NN_Model.fit(group_df_3,y_3)
nn_4 = NN_Model.fit(group_df_4,y_4)
nn_5 = NN_Model.fit(group_df_5,y_5)
nn_6 = NN_Model.fit(group_df_6,y_6)

delta = time.time() - start
print ("model trained finished in %.2f seconds" % (delta,))

# Filtering part of recommended tracks

In [4]:
raw_recommend = pd.read_csv('/Users/osx/Desktop/Warner Music Dissertation/Code/test_supervised.csv')
raw_recommend = raw_recommend.drop('Unnamed: 0',axis=1)
raw_recommend.sort_values('customer_id',inplace=True)
raw_recommend.reset_index(drop=True,inplace=True)

In [5]:
age = pd.DataFrame(2017-raw_recommend['birth_year'].values)
raw_recommend = pd.concat([raw_recommend,age],axis=1)
raw_recommend.rename(columns={0:'age'},inplace=True)
raw_recommend.drop('birth_year',axis=1,inplace=True)

In [6]:
raw_recommend.head(5)

,customer_id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,artist_name,album_name,age
0,00000255e47132304859a31172d40a34,Bloodstream,0.48400,0.559,306000.0,0.504,0.000048,3.0,0.1180,-10.019,0.0,0.0367,159.751,4.0,0.261,Ed Sheeran,x,26
1,00000255e47132304859a31172d40a34,Sunday Sunday - 2012 Remastered Version,0.00318,0.342,158360.0,0.857,0.000000,7.0,0.3540,-5.985,1.0,0.0456,118.645,4.0,0.849,Blur,Modern Life Is Rubbish [Special Edition],26
2,00000255e47132304859a31172d40a34,Take It Back,0.01730,0.805,348907.0,0.739,0.000073,6.0,0.1080,-6.712,1.0,0.0443,124.031,4.0,0.554,Ed Sheeran,x,26
3,00000255e47132304859a31172d40a34,Mack The Knife,0.13900,0.738,373427.0,0.800,0.000399,5.0,0.0593,-6.232,1.0,0.0783,109.966,4.0,0.742,Michael Bublé,Special Delivery,26
4,00000255e47132304859a31172d40a34,Revolver - feat. Lil Wayne,0.01890,0.736,220280.0,0.847,0.105000,10.0,0.0951,-4.637,0.0,0.0549,123.952,4.0,0.757,Madonna,Celebration,26


In [19]:
k=0
for j in tqdm.tqdm([group_1,group_2,group_3,group_4,group_5,group_6]):
    k+=1
    group_list = []
    for i in j:
        group_list.append(customer[i][0])
    if k==1:
        test_1 = raw_recommend[raw_recommend.customer_id.isin(group_list)]
    if k==2:
        test_2 = raw_recommend[raw_recommend.customer_id.isin(group_list)]
    if k==3:
        test_3 = raw_recommend[raw_recommend.customer_id.isin(group_list)]
    if k==4:
        test_4 = raw_recommend[raw_recommend.customer_id.isin(group_list)]
    if k==5:
        test_5 = raw_recommend[raw_recommend.customer_id.isin(group_list)]
    if k==6:
        test_6 = raw_recommend[raw_recommend.customer_id.isin(group_list)]
#     if k==7:
#         group_df_7 = train[train.customer_id.isin(group_list)]
#     if k==8:
#         group_df_8 = train[train.customer_id.isin(group_list)]
#     if k==9:
#         group_df_9 = train[train.customer_id.isin(group_list)]
#     if k==10:
#         group_df_10 = train[train.customer_id.isin(group_list)]

100%|██████████| 6/6 [00:00<00:00, 24.31it/s]


In [20]:
raw_recommend.columns

Index(['customer_id', 'track_name', 'acousticness', 'danceability',
       'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness',
       'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence',
       'artist_name', 'album_name', 'age'],
      dtype='object')

In [21]:
features_to_drop = ['customer_id', 'track_name','artist_name', 'album_name']
raw_recommend_train = raw_recommend.drop(features_to_drop,axis=1)
test_1 = test_1.drop(features_to_drop,axis=1)
test_2 = test_2.drop(features_to_drop,axis=1)
test_3 = test_3.drop(features_to_drop,axis=1)
test_4 = test_4.drop(features_to_drop,axis=1)
test_5 = test_5.drop(features_to_drop,axis=1)
test_6 = test_6.drop(features_to_drop,axis=1)

In [22]:
v_ft=np.load('validation_Dict.npy').item()
active_user = np.load('active_user.npy')
t = np.load('trainset_Dict.npy').item()
index_to_user_dict = np.load('train_user_map.npy').item()
index_to_track_id = np.load('train_track_map.npy').item()
inv_track_id_map = np.load('inv_track_map.npy').item()
track_uni_=np.load('track_feature_1.npy').item()

##### xgboost filtering

In [39]:
filename = '/Users/osx/Desktop/Warner Music Dissertation/Model/xgboost.sav'
xgboost = pickle.load(open(filename, 'rb'))

In [44]:
y_filter = xgboost.predict(raw_recommend_train)

In [59]:
pred = pd.concat([raw_recommend[['customer_id','track_name']],pd.DataFrame(y_filter).rename(columns={0:'pre_ratings'})]\
                 ,axis=1)

In [63]:
pred['pre_ratings'].describe()

count    375000.000000
mean          1.326602
std           1.268441
min           1.099403
25%           1.264710
50%           1.289279
75%           1.316063
max          58.952877
Name: pre_ratings, dtype: float64

In [68]:
user, index = np.unique(pred['customer_id'],True)
rec_track = np.split(pred['track_name'],index[1:])
pre_track = np.split(pred['pre_ratings'],index[1:])

In [129]:
keep_ = 15
filtered_rec = {}
for i in tqdm.tqdm(range(15000)):
    filtered_rec[i] = rec_track[i][pre_track[i].nlargest(keep_).index.tolist()].values.tolist()

100%|██████████| 15000/15000 [00:16<00:00, 924.09it/s]


In [153]:
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set = filtered_rec[i]
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall_xgb = suc_rec/Tnum_val
precision_xgb = suc_rec/num_p
print('xgboost without k-means filtered recall is',recall_xgb)
print('xgboost without k-means filtered precision is',precision_xgb)

xgboost without k-means filtered recall is 0.03859758444367062
xgboost without k-means filtered precision is 0.01198872854581163


##### Neural Network with propogation filtering

In [131]:
filename = '/Users/osx/Desktop/Warner Music Dissertation/Model/NeuralNetwork.sav'
NN = pickle.load(open(filename, 'rb'))

In [132]:
y_NN = NN.predict(raw_recommend_train)

In [133]:
pred_NN = pd.concat([raw_recommend[['customer_id','track_name']],pd.DataFrame(y_NN).rename(columns={0:'pre_ratings'})]\
                 ,axis=1)

In [134]:
user, index = np.unique(pred_NN['customer_id'],True)
rec_track = np.split(pred_NN['track_name'],index[1:])
pre_track = np.split(pred_NN['pre_ratings'],index[1:])

In [139]:
keep_ = 15
NN_filtered_rec = {}
for i in tqdm.tqdm(range(15000)):
    NN_filtered_rec[i] = rec_track[i][pre_track[i].nlargest(keep_).index.tolist()].values.tolist()

100%|██████████| 15000/15000 [00:14<00:00, 1032.60it/s]


In [154]:
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set = NN_filtered_rec[i]
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall_nn = suc_rec/Tnum_val
precision_nn = suc_rec/num_p
print('Neural Network without k-means filtered recall is',recall_nn)
print('Neural Network without k-means filtered precision is',precision_nn)

Neural Network without k-means filtered recall is 0.03273280428128952
Neural Network without k-means filtered precision is 0.010167079384054877


##### Random Forest filter

In [77]:
filename = '/Users/osx/Desktop/Warner Music Dissertation/Model/randaomforest.sav'
RFR = pickle.load(open(filename, 'rb'))

In [78]:
y_RFR = RFR.predict(raw_recommend_train)

In [79]:
pred_RFR = pd.concat([raw_recommend[['customer_id','track_name']],pd.DataFrame(y_RFR).rename(columns={0:'pre_ratings'})]\
                 ,axis=1)

In [147]:
user, index = np.unique(pred_RFR['customer_id'],True)
rec_track = np.split(pred_RFR['track_name'],index[1:])
pre_track = np.split(pred_RFR['pre_ratings'],index[1:])

In [148]:
keep_ = 15
RFR_filtered_rec = {}
for i in tqdm.tqdm(range(15000)):
    RFR_filtered_rec[i] = rec_track[i][pre_track[i].nlargest(keep_).index.tolist()].values.tolist()

100%|██████████| 15000/15000 [00:14<00:00, 1024.18it/s]


In [152]:
len(RFR_filtered_rec[0])

15

In [155]:
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set = RFR_filtered_rec[i]
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall_rfr = suc_rec/Tnum_val
precision_rfr = suc_rec/num_p
print('Random Forest without k-means filtered recall is',recall_rfr)
print('Random Forest without k-means filtered precision is',precision_rfr)

Random Forest without k-means filtered recall is 0.03683815039495629
Random Forest without k-means filtered precision is 0.011442233797284604


##### SVM filtering

### with K-means

###### neruak network

In [22]:
# neural network
filename = '/Users/osx/Desktop/Warner Music Dissertation/Model/nn1.sav'
nn1 = pickle.load(open(filename, 'rb'))
filename = '/Users/osx/Desktop/Warner Music Dissertation/Model/nn2.sav'
nn2 = pickle.load(open(filename, 'rb'))
filename = '/Users/osx/Desktop/Warner Music Dissertation/Model/nn3.sav'
nn3 = pickle.load(open(filename, 'rb'))
filename = '/Users/osx/Desktop/Warner Music Dissertation/Model/nn4.sav'
nn4 = pickle.load(open(filename, 'rb'))
filename = '/Users/osx/Desktop/Warner Music Dissertation/Model/nn5.sav'
nn5 = pickle.load(open(filename, 'rb'))
filename = '/Users/osx/Desktop/Warner Music Dissertation/Model/nn6.sav'
nn6 = pickle.load(open(filename, 'rb'))

In [83]:
y_nn1 = nn1.predict(test_1)
y_nn2 = nn2.predict(test_2)
y_nn3 = nn3.predict(test_3)
y_nn4 = nn4.predict(test_4)
y_nn5 = nn5.predict(test_5)
y_nn6 = nn6.predict(test_6)

In [99]:
pred_1 = pd.DataFrame(y_nn1,index=test_1.index).rename(columns={0:'pre_ratings'})
pred_2 = pd.DataFrame(y_nn2,index=test_2.index).rename(columns={0:'pre_ratings'})
pred_3 = pd.DataFrame(y_nn3,index=test_3.index).rename(columns={0:'pre_ratings'})
pred_4 = pd.DataFrame(y_nn4,index=test_4.index).rename(columns={0:'pre_ratings'})
pred_5 = pd.DataFrame(y_nn5,index=test_5.index).rename(columns={0:'pre_ratings'})
pred_6 = pd.DataFrame(y_nn6,index=test_6.index).rename(columns={0:'pre_ratings'})

In [101]:
pred = pd.concat([pred_1,pred_2,pred_3,pred_4,pred_5,pred_6])
pred = pred.reset_index().sort_values('index').reset_index()
pred.drop(['level_0','index'],axis=1,inplace=True)
pred_NN = pd.concat([raw_recommend[['customer_id','track_name']],pred],axis=1)

In [113]:
user, index = np.unique(pred_NN['customer_id'],True)
rec_track = np.split(pred_NN['track_name'],index[1:])
pre_track = np.split(pred_NN['pre_ratings'],index[1:])

In [114]:
keep_ = 15
NN_filtered_rec = {}
for i in tqdm.tqdm(range(15000)):
    NN_filtered_rec[i] = rec_track[i][pre_track[i].nlargest(keep_).index.tolist()].values.tolist()

100%|██████████| 15000/15000 [00:13<00:00, 1073.00it/s]


In [115]:
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set = NN_filtered_rec[i]
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall_nn = suc_rec/Tnum_val
precision_nn = suc_rec/num_p
print('Neural Network with k-means filtered recall is',recall_nn)
print('Neural Network with k-means filtered precision is',precision_nn)

Neural Network with k-means filtered recall is 0.032714476843282075
Neural Network with k-means filtered precision is 0.010161386730424387


###### xgboost

In [3]:
# neural network
filename = '/Users/osx/Desktop/Warner Music Dissertation/Model/xgboost_1.sav'
xgb_1 = pickle.load(open(filename, 'rb'))
filename = '/Users/osx/Desktop/Warner Music Dissertation/Model/xgboost_2.sav'
xgb_2 = pickle.load(open(filename, 'rb'))
filename = '/Users/osx/Desktop/Warner Music Dissertation/Model/xgboost_3.sav'
xgb_3 = pickle.load(open(filename, 'rb'))
filename = '/Users/osx/Desktop/Warner Music Dissertation/Model/xgboost_4.sav'
xgb_4 = pickle.load(open(filename, 'rb'))
filename = '/Users/osx/Desktop/Warner Music Dissertation/Model/xgboost_5.sav'
xgb_5 = pickle.load(open(filename, 'rb'))
filename = '/Users/osx/Desktop/Warner Music Dissertation/Model/xgboost_6.sav'
xgb_6 = pickle.load(open(filename, 'rb'))

In [23]:
y_xgb1 = xgb_1.predict(test_1)
y_xgb2 = xgb_2.predict(test_2)
y_xgb3 = xgb_3.predict(test_3)
y_xgb4 = xgb_4.predict(test_4)
y_xgb5 = xgb_5.predict(test_5)
y_xgb6 = xgb_6.predict(test_6)

In [24]:
pred_1 = pd.DataFrame(y_xgb1,index=test_1.index).rename(columns={0:'pre_ratings'})
pred_2 = pd.DataFrame(y_xgb2,index=test_2.index).rename(columns={0:'pre_ratings'})
pred_3 = pd.DataFrame(y_xgb3,index=test_3.index).rename(columns={0:'pre_ratings'})
pred_4 = pd.DataFrame(y_xgb4,index=test_4.index).rename(columns={0:'pre_ratings'})
pred_5 = pd.DataFrame(y_xgb5,index=test_5.index).rename(columns={0:'pre_ratings'})
pred_6 = pd.DataFrame(y_xgb6,index=test_6.index).rename(columns={0:'pre_ratings'})

In [26]:
pred = pd.concat([pred_1,pred_2,pred_3,pred_4,pred_5,pred_6])
pred = pred.reset_index().sort_values('index').reset_index()
pred.drop(['level_0','index'],axis=1,inplace=True)
pred_XGB = pd.concat([raw_recommend[['customer_id','track_name']],pred],axis=1)

In [27]:
user, index = np.unique(pred_XGB['customer_id'],True)
rec_track = np.split(pred_XGB['track_name'],index[1:])
pre_track = np.split(pred_XGB['pre_ratings'],index[1:])

In [28]:
keep_ = 15
XGB_filtered_rec = {}
for i in tqdm.tqdm(range(15000)):
    XGB_filtered_rec[i] = rec_track[i][pre_track[i].nlargest(keep_).index.tolist()].values.tolist()

100%|██████████| 15000/15000 [00:15<00:00, 959.47it/s]


In [29]:
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set = XGB_filtered_rec[i]
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall_nn = suc_rec/Tnum_val
precision_nn = suc_rec/num_p
print('XGB with k-means filtered recall is',recall_nn)
print('XGB with k-means filtered precision is',precision_nn)

XGB with k-means filtered recall is 0.02831589172149625
XGB with k-means filtered precision is 0.008795149859106823


###### Random forest 